Deep Learning
=============

Assignment 5
------------

The goal of this assignment is to train a Word2Vec skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function

import collections
import datetime
import math
import os
import random
import zipfile

import numpy as np
import tensorflow as tf
import tqdm
from matplotlib import pylab
from sklearn.manifold import TSNE

from six.moves import range
from six.moves.urllib.request import urlretrieve

/home/dominik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Download the data from the source website if necessary.

In [2]:
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename +
                        '. Can you get to it with a browser?')
    return filename


filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


Read the data into a string.

In [3]:
words = []
for path in tqdm.tqdm_notebook(os.listdir('./textbase-optimized/')[:500]):
    try:
        with open('./textbase-optimized/' + path, 'r') as f:
            string = f.read().lower()
            
            translation_table = dict.fromkeys(map(ord, '!@#$",.?“”()_;'), None)
            filtered_string = string.translate(translation_table)
            
            words.extend(filtered_string.split())

    except UnicodeDecodeError:
        print('UnicodeDecodeError in file: ' + path)

        

def read_data(filename):
    # Extract the first file enclosed in a zip file as a list of words
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


words.extend(read_data(filename))
print(words[:20])
print('Data size %d' % len(words))

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

UnicodeDecodeError in file: Henry David Thoreau___A Week on the Concord and Merrimack Rivers.txt

['the', 'stokesley', 'secret', 'by', 'charlotte', 'm', 'yonge', 'chapter', 'i', 'how', 'can', 'a', 'pig', 'pay', 'the', 'rent', 'the', 'question', 'seemed', 'to']
Data size 50919413


In [4]:
print(' '.join(words[1000000:1000100]))

and you see there is nothing to keep me over here any longer tell me have you had any fortune yet i read the papers every day hoping to see that you have cleared up those two terrible affairs inspector jacks shook his head not yet prince he said not yet the prince echoed dear me that is very unfortunate inspector jacks watched the people who were passing for a moment with a fixed unseeing gaze i am afraid he said that we must seem to you very slow and very stupid very likely we are and yet yet in


Build the dictionary and replace rare words with UNK token.

In [5]:
vocabulary_size = 50000


def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0

    for word in tqdm.tqdm_notebook(words):
        if word in dictionary:
            data.append(dictionary[word])
        else:
            data.append(0)  # dictionary['UNK']

            unk_count = unk_count + 1

    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary


data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
#del words  # Hint to reduce memory.

HBox(children=(IntProgress(value=0, max=50919413), HTML(value='')))


Most common words (+UNK) [['UNK', 1400263], ('the', 3046802), ('of', 1645713), ('and', 1524954), ('to', 1245800)]
Sample data [1, 0, 840, 21, 7283, 348, 34990, 511, 10, 118]


In [6]:
with open('./tf_log/metadata.tsv', 'w+') as f:
    f.write('Word\tFrequency\n')
    f.write('\n'.join([label+'\t'+str(occurrences) for label, occurrences in count]))

In [9]:
print(words[:30])  # The original text corpus (separated into words)

['THE', 'STOKESLEY', 'SECRET', 'by', 'Charlotte', 'M.', 'Yonge', 'CHAPTER', 'I.', '"How', 'can', 'a', 'pig', 'pay', 'the', 'rent?"', 'The', 'question', 'seemed', 'to', 'have', 'been', 'long', 'under', 'consideration,', 'to', 'judge', 'by', 'the', 'manner']


In [34]:
print(
    data[:70]
)  # The original text corpus but words are replaced with their indices and rare words use index 0 (UNK)

[5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103, 855, 3, 1, 15068, 0, 2, 1, 151, 855, 3581, 1, 195, 11, 191, 59, 5, 6, 10713, 215, 7, 1325, 105, 455, 20, 59, 2732, 363, 7, 3673, 1, 709, 2, 372, 27, 41, 37, 54, 540, 98, 12, 6, 1424, 2758, 19, 568, 687, 7089, 1, 248, 5234]


In [35]:
print(
    count[:20]
)  # Tuples with the 50000 (vocabulary_size) most common words and their counts

[['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430), ('two', 192644), ('is', 183153), ('as', 131815), ('eight', 125285), ('for', 118445), ('s', 116710), ('five', 115789), ('three', 114775), ('was', 112807), ('by', 111831)]


In [36]:
print(list(dictionary.items())[:25])  # dict with all words and their indices

[('UNK', 0), ('the', 1), ('of', 2), ('and', 3), ('one', 4), ('in', 5), ('a', 6), ('to', 7), ('zero', 8), ('nine', 9), ('two', 10), ('is', 11), ('as', 12), ('eight', 13), ('for', 14), ('s', 15), ('five', 16), ('three', 17), ('was', 18), ('by', 19), ('that', 20), ('four', 21), ('six', 22), ('seven', 23), ('with', 24)]


In [37]:
print(list(reverse_dictionary.items())
      [:25])  # dict with all words and their indices (reverse lookup)

[(0, 'UNK'), (1, 'the'), (2, 'of'), (3, 'and'), (4, 'one'), (5, 'in'), (6, 'a'), (7, 'to'), (8, 'zero'), (9, 'nine'), (10, 'two'), (11, 'is'), (12, 'as'), (13, 'eight'), (14, 'for'), (15, 's'), (16, 'five'), (17, 'three'), (18, 'was'), (19, 'by'), (20, 'that'), (21, 'four'), (22, 'six'), (23, 'seven'), (24, 'with')]


Function to generate a training batch for the skip-gram model.

In [7]:
data_index = 0


def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    #data_index = 0
    batch, labels = generate_batch(
        batch_size=16, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips,
                                                           skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(16)])
    
    print(batch)
    print(labels)

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term', 'of', 'of', 'abuse', 'abuse', 'first', 'first', 'used', 'used']
    labels: ['anarchism', 'as', 'originated', 'a', 'as', 'term', 'a', 'of', 'abuse', 'term', 'of', 'first', 'abuse', 'used', 'first', 'against']
[3081 3081   12   12    6    6  195  195    2    2 3134 3134   46   46
   59   59]
[[5234]
 [  12]
 [3081]
 [   6]
 [  12]
 [ 195]
 [   6]
 [   2]
 [3134]
 [ 195]
 [   2]
 [  46]
 [3134]
 [  59]
 [  46]
 [ 156]]

with num_skips = 4 and skip_window = 2:
    batch: ['radicals', 'radicals', 'radicals', 'radicals', 'including', 'including', 'including', 'including', 'the', 'the', 'the', 'the', 'diggers', 'diggers', 'diggers', 'diggers']
    labels: ['the', 'including', 'working', 'class', 'diggers', 'class', 'radicals', 'the', 'diggers', 'radicals', 'including', 'of', 'including', 'the', 'the

Train a skip-gram model.

In [44]:
batch_size = 128
embedding_size = 250  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64  # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name='word_embedding')
    
    softmax_weights = tf.Variable(
        tf.truncated_normal(
            [vocabulary_size, embedding_size],
            stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(
            weights=softmax_weights,
            biases=softmax_biases,
            inputs=embed,
            labels=train_labels,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))
    
    tf.summary.scalar('loss', loss)

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(1.0, global_step,
                                               100000, 0.96, staircase=True)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                              valid_dataset)
    similarity = tf.matmul(valid_embeddings,
                           tf.transpose(normalized_embeddings))

In [45]:
num_steps = 150001

with tf.Session(graph=graph) as session:
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(
        './tf_log/' + datetime.datetime.now().strftime('%H:%M:%S'), session.graph)
    merged = tf.summary.merge_all()
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in tqdm.tqdm_notebook(range(num_steps)):
        batch_data, batch_labels = generate_batch(batch_size, num_skips,
                                                  skip_window)
        feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
        _, l, summary = session.run([optimizer, loss, merged], feed_dict=feed_dict)
        writer.add_summary(summary, step)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
            saver.save(session, os.path.join('./tf_log/', "model.ckpt"), step)
    final_embeddings = normalized_embeddings.eval()

Initialized


HBox(children=(IntProgress(value=0, max=150001), HTML(value='')))

Average loss at step 0: 7.371177
Nearest to only: least,", Jacobites, mastery, working., Tom,, tents, Vane., shrugging,
Nearest to have: "Gee!, sovereign, "Poetical, handiwork, nettled, sensations., sentiment,, shrieked,
Nearest to a: contention, Anty, secretariat, sox, appointing, duel,, 155, UNDER,
Nearest to from: compass, postulates, dew,, jihad, nordic, hazardous, Chuck, Phantom,
Nearest to said: chagrin, shirt., mick, renewing, Joyce,, individuals, ‘What, stein,
Nearest to And: homes., detail., snooker, toy, rare,, arf, Charles's, incredulous,
Nearest to eight: tip, joystick, huxley, footprint, madeira, disturbing, soldier;, olympics,
Nearest to can: feelings., club, grove, fold,, fall,, butterfly,, alcoholism, justices,
Nearest to so: markers, treasonable, bronx, roadside,, straightened, ois, japan, expressly,
Nearest to could: neighbor's, despising, is,, is!, sitting, hurt?", ransacked, jars,
Nearest to should: treasury, proclaimed, advert, rectify, submission., publications, N

KeyboardInterrupt: 

In [ ]:
num_points = 1500

tsne = TSNE(
    perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points + 1, :])

In [ ]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15, 15))  # in inches
    for i, label in enumerate(labels):
        x, y = embeddings[i, :]
        pylab.scatter(x, y)
        pylab.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')
    pylab.show()


words = [reverse_dictionary[i] for i in range(1, num_points + 1)]
plot(two_d_embeddings, words)

---

Problem
-------

An alternative to skip-gram is another Word2Vec model called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

---

In [7]:
data_index = 2


def generate_cbow_batch(batch_size):
    global data_index
    batch = np.ndarray(shape=(batch_size, 4), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    batch_index = 0
    while batch_index < batch_size:
        try:
            batch[batch_index][0] = data[data_index - 2]
            batch[batch_index][1] = data[data_index - 1]
            labels[batch_index][0] = data[data_index]
            batch[batch_index][2] = data[data_index + 1]
            batch[batch_index][3] = data[data_index + 2]
            batch_index += 1
            data_index = (data_index + 1) % len(data)
        except IndexError:
            pass

    return batch, labels


print('data:', [reverse_dictionary[di] for di in data[data_index-2:data_index+10]])

for v in range(4):

    batch, labels = generate_cbow_batch(batch_size=1)
    print()
    print([[reverse_dictionary[y] for y in x] for x in batch])
    print([reverse_dictionary[x[0]] for x in labels])
    

    #print('    batch:', [reverse_dictionary[bi] for bi in batch])
    #print('    labels:', [reverse_dictionary[li] for li in labels.reshape(16)])

    #print(batch)
    #print(labels)

data: ['the', 'UNK', 'secret', 'by', 'charlotte', 'm', 'yonge', 'chapter', 'i', 'how', 'can', 'a']

[['the', 'UNK', 'by', 'charlotte']]
['secret']

[['UNK', 'secret', 'charlotte', 'm']]
['by']

[['secret', 'by', 'm', 'yonge']]
['charlotte']

[['by', 'charlotte', 'yonge', 'chapter']]
['m']


In [10]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64  # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size, 4])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name='word_embedding')
    
    
    softmax_weights = tf.Variable(
        tf.truncated_normal(
            [vocabulary_size, embedding_size],
            stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed_lookup = tf.nn.embedding_lookup(embeddings, train_dataset)
    
    bow_vector = tf.reduce_sum(embed_lookup, axis=1)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(
            weights=softmax_weights,
            biases=softmax_biases,
            inputs=bow_vector,
            labels=train_labels,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))
    
    tf.summary.scalar('loss', loss)

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(1.0, global_step,
                                               100000, 0.932, staircase=True)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                              valid_dataset)
    similarity = tf.matmul(valid_embeddings,
                           tf.transpose(normalized_embeddings))

In [11]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(
        './tf_log/' + datetime.datetime.now().strftime('%H:%M:%S'), session.graph)
    merged = tf.summary.merge_all()
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in tqdm.tqdm_notebook(range(num_steps)):
        batch_data, batch_labels = generate_cbow_batch(batch_size=batch_size)
        feed_dict = {train_dataset: batch_data, train_labels: batch_labels}
        _, l, summary = session.run([optimizer, loss, merged], feed_dict=feed_dict)
        writer.add_summary(summary, step)
        average_loss += l
        if step % 5000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
            saver.save(session, os.path.join('./tf_log/', "model.ckpt"), step)
    final_embeddings = normalized_embeddings.eval()

Initialized


HBox(children=(IntProgress(value=0, max=100001), HTML(value='')))

Average loss at step 0: 8.105158
Nearest to four: myrrh, warningly, experiencing, sampled, bulging, algardi, syrian, creaking,
Nearest to six: potato, gand, swim, whiles, umm, endeavoured, aorta, trustworthy,
Nearest to up: callum, stanley, magrath, jesse, configurations, tyee, tubes, blinked,
Nearest to there: westwood, garnet, imaginable, saddle, entrepreneurial, stirling, subside, style,
Nearest to did: heme, cora, bees, bhp, unrestrained, legislation, alkanes, smith's,
Nearest to they: cynics, prominently, tugging, morgan's, hounded, synonyms, pleasantry, pollux,
Nearest to well: aunty, arbitrator, deformed, arbour, est, sparta, diagnosis, soundness,
Nearest to not: balmoral, hackit, coiffure, nineties, mannered, a:, swell, prosper,
Nearest to that: discrete, fault, defender, waz-don, creationism, momentary, supposedly, betsey,
Nearest to all: babur, toed, distinctness, absolution, goal, madness, laurels, dictation,
Nearest to would: incarcerated, embattled, scouring, redolent, leo

Average loss at step 65000: 6.975465
Average loss at step 70000: 7.475056
Nearest to four: two, three, six, several, ten, five, twenty, thirty,
Nearest to six: four, eight, ten, twelve, three, five, several, twenty,
Nearest to up: forward, off, down, away, back, along, carvil, around,
Nearest to there: longer, here, it, pollock, silent, there's, banff, born,
Nearest to did: does, could, do, didn't, will, would, should, shall,
Nearest to they: we, you, i, she, he, 'you, even, people,
Nearest to well: soon, fast, just, cahiers, much, fellers, often, punic,
Nearest to not: never, hardly, encircled, scarcely, also, always, indeed, romano,
Nearest to that: which, because, ainslie, what, where, cute, this, disruptive,
Nearest to all: both, studded, delusive, forbes, various, nuptial, everything, whilk,
Nearest to would: might, will, may, must, should, could, i'll, shall,
Nearest to these: those, qadhafi, coverts, such, several, sorts, hundred, disarranged,
Nearest to when: till, until, befor

In [ ]:
num_points = 1000

tsne = TSNE(
    perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points + 1, :])

In [ ]:
def plot(embeddings, labels):
    assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
    pylab.figure(figsize=(15, 15))  # in inches
    for i, label in enumerate(labels):
        x, y = embeddings[i, :]
        pylab.scatter(x, y)
        pylab.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')
    pylab.show()


words = [reverse_dictionary[i] for i in range(1, num_points + 1)]
plot(two_d_embeddings, words)

In [ ]:
print('done!!')